In [1]:
! pip install transformers trl accelerate torch bitsandbytes peft datasets -qU


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from datasets import load_dataset
import pandas as pd

In [4]:
df = pd.read_csv('./cards-cut2.csv',on_bad_lines='skip')

In [5]:
df['propmt'] = "give me random hearthstone card"
df['summary'] = df.apply(lambda row: f"""class:{row['cardClass']},cost:{row['cost']},flavor:{row['flavor']},name:{row['name']},rarity:{row['rarity']},spellSchool:{row['spellSchool']},text:{row['text']},type:{row['type']},attack:{row['attack']},health:{row['health']},races:{row['races']},durability:{row['durability']}   """, axis=1)

In [6]:
df.head(3)

,cardClass,cost,dbfId,flavor,id,name,rarity,spellSchool,text,type,attack,health,races,durability,propmt,summary
0,MAGE,5.0,2539,"It's on the rack next to ice lance, acid lance...",AT_001,Flame Lance,COMMON,FIRE,Deal $25 damage\nto a minion.,SPELL,NaN,NaN,NaN,NaN,give me random hearthstone card,"class:MAGE,cost:5.0,flavor:It's on the rack ne..."
1,MAGE,3.0,2541,"Burning man, brah.",AT_002,Effigy,RARE,FIRE,"<b>Secret:</b> When a friendly minion dies, su...",SPELL,NaN,NaN,NaN,NaN,give me random hearthstone card,"class:MAGE,cost:3.0,flavor:Burning man, brah.,..."
2,MAGE,2.0,2545,And he can't get up.,AT_003,Fallen Hero,RARE,NaN,Your Hero Power deals 1 extra damage.,MINION,3.0,2.0,['UNDEAD'],NaN,give me random hearthstone card,"class:MAGE,cost:2.0,flavor:And he can't get up..."


In [7]:
from datasets import load_dataset, DatasetDict, Dataset
x_train = df['propmt']
y_train = df['summary']
dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train})})

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample['text'].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample['label']
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

In [13]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [19]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="no",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 6425
    })
})

In [20]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `S

Generating train split: 0 examples [00:00, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:432: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [21]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   4763 MiB |  13090 MiB |  16146 GiB |  16141 GiB |\n|       from large pool |   4344 MiB |  12664 MiB |  16123 GiB |  16119 GiB |\n|       from small pool |    419 MiB |    627 MiB |     22 GiB |     22 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   4763 MiB |  13090 MiB |  16146 GiB |  16141 GiB |\n|       from large pool |   4344 MiB |  12664 MiB |

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.694800
20,0.654300
30,0.668600
40,0.668100
50,0.646900
60,0.632400
70,0.616200
80,0.640600
90,0.567500
100,0.619000


TrainOutput(global_step=100, training_loss=0.6408410453796387, metrics={'train_runtime': 1116.6104, 'train_samples_per_second': 0.358, 'train_steps_per_second': 0.09, 'total_flos': 3.50843194834944e+16, 'train_loss': 0.6408410453796387, 'epoch': 0.8064516129032258})

In [24]:
trainer.save_model("hs_gen_mistral_instruct")

In [ ]:
model.save_pretrained("./model.pt")

In [27]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [28]:
generate_response("### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\n--give me random hearthstone card.\n\n### Response:", model)

KeyboardInterrupt: 

In [ ]:
!pip install huggingface-hub -qU

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("exnrt/exnrt_mistral_instruct")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
model = AutoModelForCausalLM.from_pretrained("./hs_gen_mistral_instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generate_response("### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\n--give me random hearthstone card.\n\n### Response:", model)